In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install caer

In [ ]:
pip install canaro

In [ ]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'/kaggle/input/asl-dataset/asl_dataset/'

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))

In [ ]:
char_dict

In [ ]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= len(char_dict):
        break
characters

In [ ]:
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)


In [ ]:
len(train)

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)


In [ ]:
featureSet = caer.normalize(featureSet)

In [ ]:
labels = to_categorical(labels, len(characters))

In [ ]:
pip install caer

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)


In [ ]:
del train
del featureSet
del labels 
gc.collect()

In [ ]:
BATCH_SIZE = 37
EPOCHS = 10

In [ ]:
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [ ]:
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters), 
                                         loss='binary_crossentropy', decay=1e-6, learning_rate=0.001, momentum=0.9,
                                         nesterov=True)

model.summary()

In [ ]:
pip install canaro

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)


In [ ]:
print(characters)


In [ ]:

test_path = r'/kaggle/input/asl-dataset/asl_dataset/9/hand1_9_bot_seg_1_cropped.jpeg'

img = cv.imread(test_path)

plt.imshow(img)
plt.show()

In [ ]:
def prepare(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, IMG_SIZE)
    image = caer.reshape(image, IMG_SIZE, 1)
    return image



In [ ]:
predictions = model.predict(prepare(img))

In [ ]:
print(characters[np.argmax(predictions[0])])

In [ ]:
import cv2 as cv
cap = cv.VideoCapture(0)
while True:
    succes,img = cap.read()
    cv.imshow('video',img)
    cv.waitkey(1)